In [7]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 15 08:57:37 2016

@author: kevin
"""
import pandas as pd
import numpy as np

# Make a data frame from the text file
df= pd.read_csv('text')
df

,Some Garbage
0,Table title
1,X Y Z
2,-1 2 3
3,4 5 6
4,BUnhc of other stuff
5,table
6,W Y
7,1.4 1.7
8,3 2.8
9,4 4


In [10]:
# Split the series apart by spaces
tabs = []
n=df.shape[0]
for i in range(0,n):
    # Get arrays
    # Need to replace tabs first with a blank space
    # Otherwise index2\t4 ends up index24
    tabs.append(df.iloc[i,0].replace("\t",' ').split(" "))
    # Get rid of blank spaces    
    tabs[i] = list(filter(None,tabs[i]))

In [11]:
# Make New DF
df2=pd.DataFrame(tabs)
df2

,0,1,2,3,4,5,6
0,Table,title,None,None,None,None,None
1,X,Y,Z,None,None,None,None
2,-1,2,3,None,None,None,None
3,4,5,6,None,None,None,None
4,BUnhc,of,other,stuff,None,None,None
5,table,None,None,None,None,None,None
6,W,Y,None,None,None,None,None
7,1.4,1.7,None,None,None,None,None
8,3,2.8,None,None,None,None,None
9,4,4,None,None,None,None,None


In [30]:
# Get tables
# check if row above is alphabet and row below is number 
# continue checking to find where the numbers end and strings begin
# Subset this portion of the data frame
# and save that to a list
# NEED TO ADD also check if column beside is number or alphabet take 
# i.e check for index names besides the table
# i.e Stuff 1.0 2.0 3.0 4.0 5.0
n=df2.shape[0]
df_list=[]

# Use n-1 to check if row ahead is a word
for i in range(1,n-1):
    if ((df2.iloc[i-1,0] is None) | (df2.iloc[i,0] is None) | (df2.iloc[i,1] is None)):
        continue
    # Checking if a word is vertically followed by a number        
    if (df2.iloc[i-1,0][0].isalpha()) & (df2.iloc[i,0].replace('-','')[0].isdigit()):
        for j in range(i+1,n):
            if (df2.iloc[j,0].isalpha()) & (df2.iloc[j-1,0].replace('-','')[0].isdigit()):
                start=i
                end=j
                break
            # If the table ends the text file            
            else:
                start=i
                end=j+1
        df3=pd.DataFrame()
        cols = df2.iloc[i-1,:]
        k=0
        for name in cols:
            df3[name]=df2.iloc[start:end,k]
            k+=1
        df_list.append(df3)
    # Checking if a word is followed by a number horizontally
    # Alpha doesn't work on index2
    # So check first letter / object of entry
    # Use n-1 to check if row ahead is a word
    # If remove third check make i in range(1,n)
    elif (df2.iloc[i,0][0].isalpha()) & (df2.iloc[i,1].replace('-','')[0].isdigit()) & (df2.iloc[i+1,0][0].isalpha()):
        if ((df2.iloc[i+1,1] is not None)) & (df2.iloc[i+1,1].replace('-','')[0].isdigit()):       
            cols= df2.iloc[i-1,:]
            
            for k in range(i,n):
                # Check if type is none if so continue
                if (df2.iloc[k,1] is not None):
                    # If word followed by number continue
                    if (k<(n-1)) & (df2.iloc[k,0][0].isalpha()) & (df2.iloc[k,1].replace('-','')[0].isdigit()):
                        continue
                    # If at end make sure the last table ends at the last row
                    elif k==(n-1):
                        start=i
                        end=k+1
                    # If word not followed by number set start and end
                    # and exit loop
                    else:
                        start=i
                        end=k
                        break
            df3=pd.DataFrame()
            for w, name in enumerate(cols.dropna()):
                df3[name]=df2.iloc[start:end,w+1]
            # If you want the row names as index names
            df3.index=df2.iloc[start:end,0]
            # Adds an extra blank row which is actually a name for the index
            # To get rid of it rename it none
            df3.index.name = None
            df_list.append(df3)

In [31]:
print("Seperated tables in list\n")
for data in df_list:
    print(data)

Seperated tables in list

    X  Y  Z   NaN
2  -1  2  3  None
3   4  5  6  None
     W    Y   NaN
7  1.4  1.7  None
8    3  2.8  None
9    4    4  None
        M  A  Q
index1  1  2  3
index2  4  5  6
      H  E  Y
dex1  7  8  9
dex2  4  3  2
   big variable something   NaN
20   1        2      None  None
21   1     None      None  None
      H  E  Y
dex1  7  8  9
dex2  4  3  2


In [32]:
# Get numbers only    
 
# Only works if no Negatives  
#df3=df[df[[0]].apply(lambda x: x[0].isdigit(),axis=1)]

# Works but function is deprecated
# Tried to_numeric but didn't work, can't handle strings

df4=df2.convert_objects(convert_numeric=True).dropna(how='all')
# Works but keeps columns where no numbers involved
# Even if they contain strings
print()
print("df4 data types: \n" + str(df4.dtypes))
want = (df4.dtypes == 'float64')
df4 = df4.loc[:,want]
print()
print("Nums Only\n")
df4


df4 data types: 
0    float64
1    float64
2    float64
3    float64
4     object
5     object
6     object
dtype: object

Nums Only



/home/kevin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,0,1,2,3
2,-1.0,2.0,3.0,NaN
3,4.0,5.0,6.0,NaN
7,1.4,1.7,NaN,NaN
8,3.0,2.8,NaN,NaN
9,4.0,4.0,NaN,NaN
13,NaN,1.0,2.0,3.0
14,NaN,4.0,5.0,6.0
17,NaN,7.0,8.0,9.0
18,NaN,4.0,3.0,2.0
20,1.0,2.0,NaN,NaN


In [35]:
# To get rid of NA's
df_list[0].dropna(how='all',axis=1)

,X,Y,Z
2,-1,2,3
3,4,5,6
